## Lab | Handling Data Imbalance in Classification Models

Begin the Modeling here

- Look critically at the dtypes of numerical and categorical columns and make changes where appropriate.
- Concatenate numerical and categorical back together again for your X dataframe. Designate the TargetB as y.
    - Split the data into a training set and a test set.
    - Split further into train_num and train_cat. Also test_num and test_cat.
    - Scale the features either by using MinMax Scaler or a Standard Scaler. (train_num, test_num)
    - Encode the categorical features using One-Hot Encoding or Ordinal Encoding. (train_cat, test_cat)
        - fit only on train data, transform both train and test
        - again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
    - Fit a logistic regression (classification) model on the training data.
    - Check the accuracy on the test data.
    
Note: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model has changed.

In [3]:
import pandas as pd
import numpy as np

1.  Look critically at the dtypes of numerical and categorical columns and make changes where appropriate.

In [4]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
target = pd.read_csv('target.csv')

In [5]:
numerical.dtypes

TCODE         int64
AGE         float64
INCOME        int64
WEALTH1       int64
HIT           int64
             ...   
AVGGIFT     float64
CONTROLN      int64
HPHONE_D      int64
RFA_2F        int64
CLUSTER2      int64
Length: 315, dtype: object

In [6]:
original_max_columns = pd.get_option('display.max_columns')
pd.set_option('display.max_columns', None)
print(numerical)

       TCODE        AGE  INCOME  WEALTH1  HIT  MALEMILI  MALEVET  VIETVETS  \
0          0  60.000000       5        9    0         0       39        34   
1          1  46.000000       6        9   16         0       15        55   
2          1  61.611649       3        1    2         0       20        29   
3          0  70.000000       1        4    2         0       23        14   
4          0  78.000000       3        2   60         1       28         9   
...      ...        ...     ...      ...  ...       ...      ...       ...   
95407      1  61.611649       5        9    0        14       36        47   
95408      1  48.000000       7        9    1         0       31        43   
95409      1  60.000000       5        9    0         0       18        46   
95410      0  58.000000       7        9    0         0       28        35   
95411      2  80.000000       5        8    3         1       33        22   

       WWIIVETS  LOCALGOV  STATEGOV  FEDGOV  WEALTH2  POP901  P

In [7]:
numerical.dtypes.value_counts()

int64      306
float64      9
dtype: int64

In [8]:
numerical = numerical.drop(columns=['CLUSTER2', 'TCODE', 'INCOME'])

In [9]:
categorical

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,27,H,M,3,L,G,C,C,2,96,1,0,2,96,2,96,2,96,2,96,2
95408,TX,24,H,M,3,L,F,A,C,1,96,1,50,1,96,3,96,3,96,3,96,3
95409,MI,30,H,M,3,L,E,B,C,3,95,1,38,1,96,3,95,1,96,10,94,10
95410,CA,24,H,F,2,L,F,A,C,1,86,1,40,5,90,11,96,8,97,1,86,12


In [10]:
categorical.dtypes

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
dtype: object

In [11]:
categorical = categorical.drop(columns=['CLUSTER'])

In [12]:
categorical = categorical.astype('object')

2. Concatenate the data and define Target as y. 

In [13]:
data = pd.concat([numerical, categorical.set_axis(numerical.index)], axis=1) # Avoid NaN generation

In [14]:
data

,AGE,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,STATE,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,60.000000,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,IL,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11
1,46.000000,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,CA,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10
2,61.611649,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6

In [135]:
data.to_csv('data.csv')

In [136]:
data = pd.read_csv('data.csv')

In [137]:
target

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
95407,0,0.0
95408,0,0.0
95409,0,0.0
95410,1,18.0


In [138]:
x = numerical
y = target['TARGET_B']

2.1. Split the data into a training set and a test set.

    - Split further into train_num and train_cat. Also test_num and test_cat.
    
    - Scale the features either by using MinMax Scaler or a Standard Scaler. (train_num, test_num)
    
    - Encode the categorical features using One-Hot Encoding or Ordinal Encoding. (train_cat, test_cat)
    
        - fit only on train data, transform both train and test
        
        - again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test

In [139]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [140]:
from sklearn.preprocessing import MinMaxScaler

train_num, test_num, train_cat, test_cat = train_test_split(x, y, test_size=0.2, random_state=2222)

scaler = MinMaxScaler()

train_num = scaler.fit_transform(train_num)
test_num = scaler.transform(test_num)

In [141]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

encoder = OrdinalEncoder()

train_cat = encoder.fit_transform(train_cat.values.reshape(-1, 1))


test_cat = encoder.transform(test_cat.values.reshape(-1, 1))


X_train = np.concatenate((train_num, train_cat), axis=1)
X_test = np.concatenate((test_num, test_cat), axis=1)


In [146]:
model = LogisticRegression(random_state = 10, solver='lbfgs')

model.fit(X_train, train_cat)

predictions = model.predict(X_test)

/home/katherine/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [147]:
train_score = model.score(X_train, train_cat)
test_score = model.score(X_test, test_cat)
print(train_score)
print(test_score)

1.0
1.0


In [144]:
# def fit_and_evaluate(X_train, train_cat, X_test, test_cat): # we create a funtion for the model beacuse we'll use it later.
#     model = LogisticRegression()
#     model.fit(X_train, train_cat)
#     y_pred = model.predict(X_test)
#     train_score = model.score(X_train, train_cat)
#     test_score = model.score(X_test, test_cat)
#     return print(train_score), print(test_score)

Overfitted?

- Checking the imbalance

In [148]:
#target.tail(40)

In [149]:
target['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [150]:
target['TARGET_D'].value_counts()

0.00     90569
10.00      941
15.00      591
20.00      577
5.00       503
         ...  
18.25        1
10.70        1
2.50         1
16.87        1
44.21        1
Name: TARGET_D, Length: 71, dtype: int64

### Oversampling/undersampling

In [175]:
y_b = target['TARGET_B']
y_d = target['TARGET_D']

In [176]:
from sklearn.utils import resample

In [177]:
# Perform upsampling and downsampling for TARGET_B
majority_b = target[target['TARGET_B'] == 0]
minority_b = target[target['TARGET_B'] == 1]

In [178]:
display(majority_b.shape)
display(minority_b.shape)

(90569, 2)

(4843, 2)

In [179]:
# Upsample minority class to match majority class (oversample)
minority_upsampled_b = resample(minority_class_b, replace=True, n_samples=len(majority_b), random_state=10)

In [180]:
display(majority_b.shape)
display(minority_upsampled_b.shape) # Now both sets are reasonable size. 

(90569, 2)

(90569, 2)

In [181]:
# Combine majority class with upsampled minority class
balanced_data_upsampled_b = pd.concat([majority_b, minority_upsampled_b])
balanced_data_upsampled_b.tail()

,TARGET_B,TARGET_D
76792,1,5.0
567,1,6.0
62644,1,20.0
53531,1,10.0
41278,1,5.0


In [190]:
train_cat_up.shape

(181138,)

In [203]:
# 1. Separar la columna de destino
y_train_up = balanced_data_upsampled_b['TARGET_B']
X_train_up = balanced_data_upsampled_b.drop('TARGET_B', axis=1)

# 2. Entrenar el modelo de regresión logística
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=10, solver='lbfgs')
model.fit(X_train_up, y_train_up)



LogisticRegression(random_state=10)

In [205]:
train_score = model.score(X_train_up, y_train_up)
# test_score = model.score(X_test, test_cat)
print(train_score)
# print(test_score)

1.0
